In [49]:
import pandas as pd
from functools import reduce

df_order_report = pd.read_csv('returns_\ca_export.csv')
df_item_export = pd.read_excel('returns_\items_export.xlsx')

#original format
# df_returns_export = pd.read_csv('returns_\\report-1714490019067.tsv', sep="\t",encoding= 'unicode_escape')
df_returns_export = pd.read_csv('returns_\\amz_export.csv',encoding= 'unicode_escape')

df_order_report = df_order_report.rename(columns={"Site Order ID": "Order ID"})
df_item_export = df_item_export.rename(columns={"Inventory Number": "SKU"})


In [ ]:
inner_join = pd.merge(
    df_returns_export,
    df_order_report,
    on='Order ID',
    how='inner'
)

final_report = pd.merge(
    inner_join,
    df_item_export,
    on='SKU',
    how='inner'
)

# final_report
# final_report.groupby('Tracking ID')['tempx'].apply(' '.join).reset_index()

final_report.to_csv(r"C:\Users\sebas\Desktop\projects\pandas\results.csv", index=False, encoding='utf-8')